## STEP 1. 형태소 분석기를 이용하여 품사에 따라 해당 단어를 추출하기

- 명사만 추출 `nouns.txt`
- 명사, 동사 추출 `nouns_verbs.txt`
- 명사, 동사, 형용사 추출 `nouns_verbs_adjectives.txt`

In [18]:
# 시간이 오래 소요되므로 파일로 저장
import os
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
input_file_path = os.getenv('HOME') + '/aiffel/weat/synopsis.txt'
output_file_path = './tokenized/nouns_verbs.txt'

'''
with open(input_file_path, 'r', encoding='utf-8') as rf:
    with open(output_file_path, 'w', encoding='utf-8') as wf:
        lines = rf.readlines()
        for line in tqdm(lines):
            words = okt.pos(line, stem=True, norm=True)
            res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
            wf.write(' '.join(res) + '\n')
'''

100%|██████████| 71156/71156 [07:09<00:00, 165.86it/s]


In [19]:
tokenized=[]
with open(output_file_path, 'r', encoding='utf-8') as rf:
    for line in rf:
        # 각 줄을 읽어 공백을 기준으로 단어를 나누고 리스트로 변환
        words = line.strip().split()
        tokenized.append(words)

In [20]:
# 개수 세기
cnt=0
for words in tokenized:
    cnt+=len(words)
print(cnt)

1723183


- Noun 개수: 1342179
- Noun & Verb 개수: 1723183
- Noun & Verb & Adjective 개수: 1850315

## STEP 2. 추출된 결과로 embedding model 만들기

In [21]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

[('작품', 0.853316068649292),
 ('다큐멘터리', 0.8298701643943787),
 ('드라마', 0.8007744550704956),
 ('영화로', 0.7821309566497803),
 ('코미디', 0.7374401688575745),
 ('가족영화', 0.7299917936325073),
 ('스토리', 0.7158835530281067),
 ('감동', 0.7140049338340759),
 ('영상', 0.7099827527999878),
 ('주제', 0.7072726488113403)]

## STEP 3. target, attribute 단어 셋 만들기

In [58]:
# TF/IDF로 해당 데이터를 가장 잘 표현하는 단어 셋 만들기
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

def save_tokens(input_file_name, output_file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+input_file_name, 'r') as fread: 
        with open('./tokenized/'+output_file_name, 'w', encoding='utf-8') as fwrite:
            print(input_file_name, '파일을 읽고 있습니다.')
            lines = fread.readlines()
            for line in tqdm(lines):
                words = okt.pos(line, stem=True, norm=True)
                res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
                fwrite.write(' '.join(res) + '\n')
                
def read_tokens(file_name):
    with open('./tokenized/' + file_name, 'r', encoding='utf-8') as fread:
        # 파일 전체를 읽어서 줄바꿈 문자를 제거하고 하나의 문자열로 반환
        tokenized = fread.read().replace('\n', ' ')
    
    return tokenized

### target 단어 추출

In [59]:
# 토큰 파일에 저장
art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'
output_art_txt = 'synopsis_art_nouns_verbs.txt'
output_gen_txt = 'synopsis_gen_nouns_verbs.txt'

'''
save_tokens(art_txt, output_art_txt)
save_tokens(gen_txt, output_gen_txt)
'''

art=read_tokens(output_art_txt)
gen=read_tokens(output_gen_txt)

In [61]:
# TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([art,gen])

In [65]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다. 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

w1_, w2_ = [], []
print('예술영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w1[i][0]]
    print(word, end=', ')
    w1_.append(word)

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w2[i][0]]
    print(word, end=', ')
    w2_.append(word)

예술영화를 대표하는 단어들:
하다, 되다, 그녀, 자신, 시작, 위해, 사랑, 않다, 사람, 늘다, 받다, 오다, 영화, 친구, 가다, 만나다, 남자, 찾다, 이다, 떠나다, 가족, 이야기, 되어다, 마을, 보다, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 빠지다, 여자, 버리다, 대한, 서로, 과연, 다시, 시간, 아들, 소녀, 보내다, 느끼다, 아내, 살다, 다른, 싶다, 모르다, 사이, 영화제, 세계, 사실, 크다, 하나, 들다, 만들다, 잃다, 점점, 남편, 감독, 어리다, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 찾아오다, 마지막, 생활, 아빠, 죽다, 지키다, 모습, 통해, 죽음, 기억, 비밀, 학교, 말다, 음악, 한편, 소년, 살아가다, 생각, 도시, 명의, 나타나다, 사고, 결혼, 전쟁, 돌아오다, 때문, 위기, 사라지다, 이제, 최고, 이자, 

일반영화를 대표하는 단어들:
하다, 되다, 자신, 그녀, 영화제, 위해, 않다, 사람, 시작, 국제, 영화, 받다, 친구, 사랑, 남자, 오다, 늘다, 가다, 이야기, 이다, 되어다, 보다, 만나다, 찾다, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 떠나다, 버리다, 다른, 들다, 마을, 시간, 엄마, 아들, 모든, 만들다, 단편, 마음, 살다, 사실, 다시, 보내다, 세계, 모습, 빠지다, 작품, 통해, 생각, 서로, 싶다, 느끼다, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 시키다, 어리다, 하나, 크다, 우리, 애니메이션, 때문, 모르다, 죽다, 여성, 주다, 말다, 죽음, 과연, 점점, 인간, 생활, 한편, 잃다, 결혼, 상황, 모두, 내다, 기억, 이르다, 나타나다, 명의, 소년, 돌아오다, 나가다, 여행, 찾아오다, 가장, 간다, 순간, 이제, 

In [63]:
# 중복 제거
n = 15

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [66]:
print('예술영화를 대표하는 단어들:')
for word in target_art:
    print(word, end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for word in target_gen:
    print(word, end=', ')

예술영화를 대표하는 단어들:
들어오다, 이름, 조제, 돌아가다, 벤자민, 한계, 그녀, 살아나다, 수상, 만남, 연애, 갈다, 지다, 지금, 비상구, 

일반영화를 대표하는 단어들:
토론토국제영화제, 조건, 리뷰, 남자, 죽다, 아침, 도움, 살인자, 비밀, 사라지다, 비밀, 그로, 여자친구, 전쟁, 샌프란시스코, 

### attribute 단어 추출 

In [ ]:
#파일 불러와서 TF-IDF 생성
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']
output_genre_txt = [txt.strip('.txt')+'nouns_verbs.txt' for txt in genre_txt]

'''
save_tokens(art_txt, output_art_txt)
save_tokens(gen_txt, output_gen_txt)
'''

genre = []
for file_name in genre_txt:
    genre.append(save_tokens(file_name))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre)

In [ ]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

## STEP 4. WEAT score 계산과 시각화

In [ ]:
from numpy import dot
from numpy.linalg import norm
# WEAT score function
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))
def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b
def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

In [ ]:
# embedding model과 단어 셋으로 WEAT score 구해보기
matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)
        
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

In [ ]:
# 시각화
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_heatmap(matrix, genre_name, title, save=True):
    np.random.seed(0)
    # 한글 지원 폰트
    sns.set(font='NanumGothic')

    # 마이너스 부호 
    plt.rcParams['axes.unicode_minus'] = False

    fig, ax = plt.subplots(figsize=(15,13))
    sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True, cmap='RdYlGn_r', ax=ax)

    ax.set_title(title, fontsize=20, pad=20)
    if save:
        plt.savefig('heatmap_'+title+'.png', bbox_inches='tight')

    plt.show()
    
plot_heatmap(matrix, genre_name, 'simple_tfidf', False)